# Setup - Execute First!
Den untenstehenden Codeblock unbedingt zu Beginn ausführen!
Dies ist nötig, damit andere Teile des Notebooks korrekt funktionieren.

In [ ]:
# Setup
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt

# Aktivieren von autom. Neuladen von externen Skripten (verhindert Caching-Probleme).
%load_ext autoreload
%autoreload 2


# Maschinenzahlen
Computer können nur endliche Anzahl an Operationen durchführen. Deshalb kann nicht jede reelle Zahl exakt dargestellt werden.

- Für eine gegebene Basis $B \in \mathbb{N}$ kann jede reelle Zahl $x \in \mathbb{R}$ wie folgt dargestellt werden:

$$\pm 0.m_{1}m_{2}m_{3}m_{4}\dots m_{n}*B^{\pm e_{1}e_{2}\dots e_{l}}$$

- **Mantisse**: $m=0.m_{1}m_{2}\dots m_{n}$
	- **Normalisierungsbedingung**: $m_1\neq0$
- **Exponent**: $e=e_{1}e_{2}e_{3}\dots e_{l}$
	- Der Exponent definiert die Position des Dezimalpunktes

Dabei gilt:

$$m_{i}, e_{i} \in \{0,1,\dots,B-1\}$$

- Man spricht von **n-Stelliger Gleitpunktarithmetik**
- $n$ = Anzahl Mantissen

# Approximations- und Rundungsfehler

- Es gibt eine obere und untere Schranke für den Exponent $e$
- $e_{min}\le e\le e_{max}$
- Bei Rechnern bsp. wenn gemäss IEEE-754 gespeichert wird:
	- $[-126; 127]$

Für eine Basis $B$ gelten folgende Schranken bei **normalisierten Gleitpunktzahlen**:

$$\begin{split}
x_{max} & = B^{e_{max}} - B^{e_{max}-n} = (1 - B^{-n})B^{e_{max}}\\
x_{min} & = B^{e_{min}-1}
\end{split}
$$

- Bei $x < x_{min}$ kommt es zu Unterlauf (underflow)
	- Es wird mit 0 weitergerechnet
- Bei $x > x_{max}$ kommt es zu Überlauf (overflow)
	- Die Rechnung wird abgebrochen

## Rundungsfehler und Maschinengenauigkeit
 

### Absoluter / Relativer Fehler
- Wenn Näherung $\widetilde{x}$ zu exaktem Wert $x$ vorhanden, dann ist Betrag der Differenz $|\widetilde{x} - x|$ der **absolute Fehler**.
- Wenn $x \neq 0 \Rightarrow |\frac{\widetilde{x} - x}{x}| = $ **relativer Fehler**

### *n-stellige* Gleichpunktarithmetik
Der Grund für Rundungsfehler ist, dass einzelne Operationen ($+,-,*,...$) auf $n+1$ Stellen genau gerechnet wird und das Ergebnis wiederum auf $n$ Stellen gerundet wird.
Es wird also nicht erst das Endergebnis, sondern jedes Zwischenergebnis gerundet. Deshalb wird jeder einzelne Rundungsfehler quasi durch die Rechnung weitergetragen. Daraus folgt die **Maschinengenauigkeit**sdefinition.

### Maschinengenauigkeit
Zahl $eps / \varepsilon /$ "Epsilon", heisst **Maschinengenauigkeit**. Bei allgemeiner Basis $B$ gilt:
$$eps := \frac{B}{2} * B^{-n} = \frac{1}{2} * B^{1-n}$$
$\Rightarrow eps$ entspricht dem **maximalen relativen Fehler**, der durch Rundung entstehen kann.
- Grösste positive Maschinenzahl, für die $1+eps=1$ gerundet wird
- Achtung: Maschinengenauigkeit $\neq$ kleinste darstellbare Zahl
- Rundung könnte zum Beispiel bei Division entstehen

#### Komplettbeispiel
- Rechner mit Mantisse $n=4$, $B=10$, Exponent $e=[-5, 5]$
- $x_{min}=B^{-5-1} = 0.000001 = 0.1 * 10^{-5}$
- $eps = \frac{10}{2}*10^{-4} = 5*10^{-4} = 0.0005$
- $eps$ wird fast erreicht bei $x = 0.10005 * 10^{5} = 10'005$
	- $rd(x) = 0.1001 * 10^{5}= 10'010$
	- $\frac{10010-10005}{10005}=0.00049975 \le 0.0005$
	- Diese Maschinenzahl könnte zum Beispiel bei einer Division zustanden gekommen sein

#### Maschinengenauigkeit von "double precision"
Für Format "double precision" in IEEE-754 gilt Mantisse $n = 53$ (hidden bit!). Für Basis $B = 2$ ist Maschinengenauigkeit in diesem Fall:
$$eps=\frac{1}{2}*2^{1-53} = 2^{-53} = 1.110223...e-16$$  



In [ ]:
from scripts.kap2.berechnung_maschinengenauigkeit import maschinengenauigkeit

# Beispiel mit Basis 10 und Präzision / Mantisse 5
basis = 10
mantisse = 5
eps = maschinengenauigkeit(basis, mantisse)
print(eps)

## Fehlerfortpflanzung bei Funktionsauswertungen / Konditionierung
- Näherung für den **absoluten Fehler bei Funktionsauswertungen**
$$[
\underbrace{\left| f(\widetilde{x}) - f(x) \right|}_{\text{absoluter Fehler von } f(x)}
\approx
\left| f'(x) \right| \cdot \underbrace{\left| \widetilde{x} - x \right|}_{\text{absoluter Fehler von } x}
]
$$
- Näherung für den **relativen Fehler bei Funktionsauswertungen**
$$
[
\underbrace{\left| \frac{f(\hat{x}) - f(x)}{f(x)} \right|}_{\text{relativer Fehler von } f(x)}
\approx
\underbrace{\left| \frac{f'(x) \cdot x}{f(x)} \right|}_{\text{Konditionszahl } K}
\cdot
\underbrace{\left| \frac{\hat{x} - x}{x} \right|}_{\text{relativer Fehler von } x}
]
$$

### Konditionszahl $K$
$$K := \frac{\left|f'(x)\right| \cdot \left|x\right|}{\left|f(x)\right|}$$
Die Konditionszahl gibt näherungsweise an, um wie viel sich der relative Fehler von $x$ bei einer Funktionsauswertung von $f(x)$ vergrösstert.

- Kleines $K\Rightarrow$ gut konditionierte Probleme.
  Der relative Fehler wird durch die Auswertung der Funktion nicht grösser.
- Grosses $K\Rightarrow$ schlecht konditionierte Probleme / "ill posed problems".
  

#### Verhalten **absoluter Fehler**:
Pflanzt sich in $x$ bei Funktionsauswertung näherungsweise mit dem Faktor $f'(x)$ fort.
$$|f'(x)| > 1 \rightarrow \text{Fehler wird grösser}$$

#### Verhalten **relativer Fehler**:
Pflanzt sich in $x$ bei Funktionsauswertung näherungsweise mit der Konditionszahl fort.
$$Konditionszahl > 1 \rightarrow \text{Fehler wird grösser}$$

#### Beispiel Subtraktion: 
**schlecht konditioniert** $\Rightarrow$ **Auslöschung**

In [ ]:
from scripts.kap2.berechnung_konditionszahl import berechne_konditionszahl
from sympy import symbols

x_symbol = symbols('x')
f = x_symbol ** 2 + 2*x_symbol + 1
x = 5 
print(berechne_konditionszahl(f, x))